In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [38]:
#load data with pandas
data=pd.read_csv('data/CrowdstormingDataJuly1st.csv')

In [39]:
data.ix[1200,:]

playerShort                toni-kroos
player                     Toni Kroos
club                   Bayern München
leagueCountry                 Germany
birthday                   04.01.1990
height                            182
weight                             78
position         Attacking Midfielder
games                               1
victories                           1
ties                                0
defeats                             0
goals                               0
yellowCards                         0
yellowReds                          0
redCards                            0
photoID                     84724.jpg
rater1                              0
rater2                              0
refNum                             66
refCountry                          4
Alpha_3                           LUX
meanIAT                      0.325185
nIAT                              127
seIAT                      0.00329681
meanExp                      0.538462
nExp        

All features for Toni Kroos - ref 66 - dyad. The column $\texttt{games}$ stands for the number of games in the player-referee dyad

We don't need all the features for our purposes. We can drop the first features including $\texttt{goals}$

In [36]:
data.drop(data.columns[:13],1,inplace=True)

In [44]:
data.ix[:19,['rater1','rater2']].T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
rater1,0.25,0.75,NaN,NaN,NaN,0.25,0.00,1.0,0.25,0.0,0.0,0.5,0.0,0.0,0.00,0.0,0.5,0.0,0.0,0.00
rater2,0.50,0.75,NaN,NaN,NaN,0.00,0.25,1.0,0.25,0.0,0.0,0.5,0.0,0.0,0.25,0.0,0.5,0.0,0.0,0.25


In [52]:
np.mean(abs(data.ix[:,'rater1']-data.ix[:,'rater2'])),np.var(abs(data.ix[:,'rater1']-data.ix[:,'rater2']))

(0.05831882267033646, 0.011343119350169325)

I've looked at a small cut of the data set and the two raters do disagree occasionally by $0.25$. Bhe overall disagreement is relativley low with $0.058\pm0.011$

In [98]:
np.count_nonzero(np.isnan(data.ix[:,'rater1']))/len(data.ix[:,'rater1'])

0.14659517352836443

$\Rightarrow$ So around $15\%$ of the instances don't have a picture attached to them. But those must not be all individual players but only individual dyads

In [108]:
np.count_nonzero(data.ix[:,'games']>40)

9

In [59]:
np.equal([0, 1, 3], np.arange(3),dtype=int)@np.array([1,1,1])

2

In [72]:
np.equal((np.array([1,2,3])+np.zeros((3,3))).T,np.ones((3,3)))

array([[ True,  True,  True],
       [False, False, False],
       [False, False, False]])

In [90]:
on=np.triu([1]*5)
np.random.seed(3)
x=np.random.randint(10,size=20).reshape((4,5))
print(on,x)

[[1 1 1 1 1]
 [0 1 1 1 1]
 [0 0 1 1 1]
 [0 0 0 1 1]
 [0 0 0 0 1]] [[8 9 3 8 8]
 [0 5 3 9 9]
 [5 7 6 0 4]
 [7 8 1 6 2]]


In [96]:
print(x[1]@on)

[ 0  5  8 17 26]


In [95]:
np.dot(x,on)

array([[ 8, 17, 20, 28, 36],
       [ 0,  5,  8, 17, 26],
       [ 5, 12, 18, 18, 22],
       [ 7, 15, 16, 22, 24]])